In [1]:
%matplotlib qt

In [2]:
import matplotlib as mpl
import sys
sys.path.append("/Users/yao/Smilei")
import happi
import numpy as np
import matplotlib.pyplot as plt

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
jet_vals[8] = [0.0, 0, 0.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

from matplotlib import font_manager

font_dirs = ['/Users/yao/Documents/Calibri and Cambria Fonts/']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

# set font
plt.rcParams['font.family'] = 'Calibri'

plt.rc('text', usetex=False)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=14)
plt.rc('legend', fontsize=12)

/var/folders/2t/97rc3fl92tg15k2l_4sk5hsh0000gn/T/ipykernel_24606/621321092.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet", YlOrRd


In [6]:
## prepare simulation data

wkdir = [
    #  '/Users/yao/Documents/Data/IFE/test0/laser_propagation_3d', # with only test particles
    #  '/Users/yao/Documents/Data/IFE/test0/laser_propagation_3d_target/', # with a thin foil target
    #  '/Users/yao/Desktop/test1/',
     '/Users/yao/Desktop/IFE_FlatTop_3D/',
     '/Users/yao/Desktop/ife_yao0/',
    ]

S0 = happi.Open(wkdir[0], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))
S1 = happi.Open(wkdir[1], reference_angular_frequency_SI = 2.0*np.pi*3e8/(1.0*1e-6))


Loaded simulation '/Users/yao/Desktop/IFE_FlatTop_3D/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics
Loaded simulation '/Users/yao/Desktop/ife_yao0/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Scanning for new particle diagnostics


In [7]:
Lx = S0.namelist.Lx / 2 / np.pi      # in um
print("Lx = ", Lx)
Ly = S0.namelist.Ly / 2 / np.pi      # in um
print("Ly = ", Ly)
Lz = S0.namelist.Lz / 2 / np.pi      # in um
print("Lz = ", Lz)

Lx =  80.0
Ly =  25.6
Lz =  25.6


In [9]:
Bxm0 = S0.Field(0,"Bx_m",subset={'z':Lz/2.},
         units=['um','fs','T'],
         vsym=True,
         cmap='smileiD',
         aspect='equal',
         )
# Bxm.set(layout='tight')
Bxm1 = S1.Field(0,"Bx_m",subset={'z':Lz/2.},
         units=['um','fs','T'],
         vsym=True,
         cmap='smileiD',
         aspect='equal',
         )

happi.multiSlide(Bxm0, Bxm1,shape=(2,1))

In [37]:
def fft_field_analysis(S, diag_num, field_name, slice_direction, slice_position, timestep, k_filter_x, k_filter_y):
    """
    Perform FFT analysis on a given field from the simulation data.
    
    Parameters:
    S : happi.Simulation
        The simulation object.
    diag_num : int
        The diagnostic number of the field.
    field_name : str
        The name of the field to analyze.
    slice_direction : str
        e.g., 'x', 'y', 'z', etc.
    slice_position : float
        The position to slice the field (e.g., Lz/2).
    timestep : int
        The timestep at which to perform the analysis.
    k_filter_x, k_filter_y : float
        The mask to filter out the signals. Laser is 1.
    
    Returns:
    kx : np.ndarray
        The wave numbers in the x-direction.
    ky : np.ndarray
        The wave numbers in the y-direction.
    spectrum : np.ndarray
        The FFT spectrum of the field.
    """
    field = S.Field(diag_num, field_name, subset={slice_direction:slice_position},
                    units=['um','fs','T'],
                    vsym=True)
    
    data = np.array(field.getData()[timestep])
    field_x = np.array(field.getAxis('x'))
    field_y = np.array(field.getAxis('y'))
    Nx = data.shape[0]
    Ny = data.shape[1]
    dx = field_x[1] - field_x[0]
    dy = field_y[1] - field_y[0]
    kx = np.fft.fftfreq(Nx, d=dx) * 2 * np.pi #spatial freq extents
    ky = np.fft.fftfreq(Ny, d=dy) * 2 * np.pi
    KX, KY = np.meshgrid(kx, ky, indexing='ij')

    pwr=12 #super gaussian power
    # k_filter_x = 0.8
    # k_filter_y = 15
    mask = np.exp(- np.sqrt((KX / k_filter_x)**2 + (KY / k_filter_y)**2)**pwr)

    data_fft  = np.fft.fft2(data, axes=(0,1))   # shape (Nx, Ny)
    Diag_data = np.real(np.fft.ifft2(data_fft* mask[:,:]))
    
    return kx, ky, data, Diag_data


In [45]:
kx0, ky0, Bx0, Bx0_masked = fft_field_analysis(S0, 0, "Bx_m", 'z', Lz/2., -1, 0.9, 15)
kx1, ky1, Bx1, Bx1_masked = fft_field_analysis(S1, 0, "Bx_m", 'z', Lz/2., -1, 0.9, 15)

In [47]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(6, 5))

im = ax.imshow(Bx0_masked.T,
               origin='lower',
               extent=[0, Lx/2, 0, Ly],
            #    vmin=-8000, vmax=8000,
               aspect='equal',
               cmap='bwr')

# Make a colorbar axis that exactly matches the height of `ax`
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)  # width & gap
cb = fig.colorbar(im, cax=cax)

ax.set_xlabel('x (um)')
ax.set_ylabel('y (um)')

plt.tight_layout()

In [13]:
S0.ParticleBinning(0,units=['um','fs','T','V/m','cm^-3'],
                   aspect='equal',
                #    data_log=True,
                   cmap='smileiD',
                   vsym=True,
                #    vmin=19,
                #    vmax=23,
                #    subset={'z':S0.namelist.Ltrans/2.},
                #    subset={'x':S0.namelist.Ltrans/2.},
                   ).slide()    


#0 - user_function of species # 1
    y from 0 to 160.85 in 128 steps 
    z from 0 to 160.85 in 128 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]



In [14]:
S0.ParticleBinning(1,units=['um','fs','T','V/m','cm^-3'],
                   aspect='equal',
                #    data_log=True,
                   cmap='smileiD',
                   vsym=True,
                #    vmin=19,
                #    vmax=23,
                #    subset={'z':S0.namelist.Ltrans/2.},
                #    subset={'x':S0.namelist.Ltrans/2.},
                   ).slide()    


#1 - user_function of species # 0
    y from 0 to 160.85 in 128 steps 
    z from 0 to 160.85 in 128 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]



In [15]:
S0.ParticleBinning(2,units=['um','fs','T','V/m','cm^-3'],
                   aspect='equal',
                #    data_log=True,
                   cmap='smileiD',
                   vsym=True,
                #    vmin=19,
                #    vmax=23,
                #    subset={'z':S0.namelist.Ltrans/2.},
                #    subset={'x':S0.namelist.Ltrans/2.},
                   ).slide()    


#2 - user_function of species # 2
    y from 0 to 160.85 in 128 steps 
    z from 0 to 160.85 in 128 steps 

The value in each bin is the sum of the `deposited_quantity` divided by the bin size and by grid_length[0]

